In [1]:
#!pip install snowflake
#!pip install "snowflake-snowpark-python[pandas]"
!pip install "snowflake-connector-python[pandas]"
!pip install snowflake-connector-python

     ---------------------------------------- 0.0/60.7 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.7 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 60.7/60.7 kB 638.3 kB/s eta 0:00:00
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)
     ---------------------------------------- 0.0/118.7 kB ? eta -:--:--
     --------- --------------------------- 30.7/118.7 kB 435.7 kB/s eta 0:00:01
     -------------------------- ------------ 81.9/118.7 kB 1.1 MB/s eta 0:00:01
     ------------------------------------ 118.7/118.7 kB 867.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.2 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 61.2/61.2 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/162.5 kB ? eta -:--:--
   ---------- ----------------------------- 41.0/162.5 kB 1.9 MB/s eta 0:00:01
   --------

In [2]:
import os
import pandas as pd
import json
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector
from pandas import json_normalize

In [3]:
# Define base do fs do projeto
base_path = '../../../'
# Define base do fs dos arquivos de input e output
relative_path_input_students = os.path.join(base_path,'datalake', 'raw','students.json')
f_students = open(relative_path_input_students)
data_students = json.load(f_students)
df_students = json_normalize(data_students['students'])
df_students.rename(columns={"grades.math" : "grade_math",	"grades.science": "grade_science", 	"grades.history" : "grade_history",	"grades.english" : "grade_english"}, inplace=True)
relative_path_input_misseddays = os.path.join(base_path,'datalake', 'raw','missed_days.json')
f_misseddays = open(relative_path_input_misseddays)
data_misseddays = json.load(f_misseddays)
df_misseddays = json_normalize(data_misseddays['missed_classes'])

In [4]:
df_students.head()

,student_id,name,grade_math,grade_science,grade_history,grade_english
0,S001,Alice Johnson,90,85,78,92
1,S002,Bob Smith,88,76,82,90
2,S003,Charlie Brown,92,89,80,94
3,S004,David Lee,78,92,85,88
4,S005,Eva Martinez,85,90,76,92


In [5]:
df_students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   student_id     50 non-null     object
 1   name           50 non-null     object
 2   grade_math     50 non-null     int64 
 3   grade_science  50 non-null     int64 
 4   grade_history  50 non-null     int64 
 5   grade_english  50 non-null     int64 
dtypes: int64(4), object(2)
memory usage: 2.5+ KB


In [6]:
df_students.shape

(50, 6)

In [7]:
df_misseddays.head()

,student_id,missed_days
0,S001,3
1,S002,5
2,S003,2
3,S004,1
4,S005,4


In [8]:
df_misseddays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   student_id   50 non-null     object
 1   missed_days  50 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 928.0+ bytes


In [9]:
df_misseddays.shape

(50, 2)

In [10]:
df_final = pd.merge(df_students, df_misseddays, how='left', on='student_id')
df_final.columns = df_final.columns.str.upper()
df_final.head()

,STUDENT_ID,NAME,GRADE_MATH,GRADE_SCIENCE,GRADE_HISTORY,GRADE_ENGLISH,MISSED_DAYS
0,S001,Alice Johnson,90,85,78,92,3.0
1,S002,Bob Smith,88,76,82,90,5.0
2,S003,Charlie Brown,92,89,80,94,2.0
3,S004,David Lee,78,92,85,88,1.0
4,S005,Eva Martinez,85,90,76,92,4.0


In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   STUDENT_ID     50 non-null     object 
 1   NAME           50 non-null     object 
 2   GRADE_MATH     50 non-null     int64  
 3   GRADE_SCIENCE  50 non-null     int64  
 4   GRADE_HISTORY  50 non-null     int64  
 5   GRADE_ENGLISH  50 non-null     int64  
 6   MISSED_DAYS    48 non-null     float64
dtypes: float64(1), int64(4), object(2)
memory usage: 2.9+ KB


In [12]:
df_final.shape

(50, 7)

In [13]:
conn = snowflake.connector.connect(
    user='',
    password='',
    url='',
    account='',
    warehouse='STUDENTS_DB',
    database='students_merged_db',
    schema='students_staging',
    role='ACCOUNTADMIN'
)

In [14]:
write_pandas(conn,df_final,table_name='FINAL_MERGED')

(True,
 1,
 50,
 [('jpbzcowqgx/file0.txt', 'LOADED', 50, 50, 1, 0, None, None, None, None)])